# AWS SAM メモ

## Implicit API （省略記法によるAPIの作成）

`AWS::Serverless::Function`のEventsに `Type: Api` と設定されていながら`RestApiId`が指定されていない場合、`AWS::ApiGateway::RestApi`が自動生成される。

こうした、明示的に書かずに省略記法で作るAPIをImplicit APIと呼ぶらしい（[参考](https://docs.aws.amazon.com/serverless-application-model/latest/developerguide/sam-specification-generated-resources-function.html#sam-specification-generated-resources-function-api)）

省略記法の場合、詳細な設定はできない。例えばEndpointTypeを変えたりCORSの設定がしたい場合は自分で `AWS::Serverless::Api` を書いたほうがいい。



## API-key認証するAPIの例

:::{dropdown} template.yml

```yaml
AWSTemplateFormatVersion: "2010-09-09"
Transform: AWS::Serverless-2016-10-31

Resources:
  # Api Gatewayの設定
  MyApi:
    Type: AWS::Serverless::Api
    Properties:
      StageName: Prod
      Auth:
        ApiKeyRequired: true # API Key認証の設定
      EndpointConfiguration:
        Type: REGIONAL # EndpointTypeを指定したい場合（任意）

  # API Keyの設定
  MyApiKey:
    Type: AWS::ApiGateway::ApiKey
    DependsOn: # API Gatewayの特定のStageが作成されてからApi Keyを作るようにする（エラー回避のため）
      - MyApiProdStage
    Properties:
      Enabled: true
      StageKeys:
        - RestApiId: !Ref MyApi
          StageName: Prod

  # Usage Planの設定。API keyでアクセスを許可するために必要
  MyApiUsagePlan:
    Type: AWS::ApiGateway::UsagePlan
    DependsOn:
      - MyApiKey
    Properties:
      ApiStages:
        - ApiId: !Ref MyApi
          Stage: Prod

  MyApiUsagePlanKey: # UsagePlanKeyはAPI keyとUsagePlanを紐づけるために必要
    Type: AWS::ApiGateway::UsagePlanKey
    DependsOn:
      - MyApiUsagePlan
    Properties:
      KeyId: !Ref MyApiKey
      KeyType: API_KEY
      UsagePlanId: !Ref MyApiUsagePlan

  # APIで呼び出すLambda Functionの設定
  MyFunction:
    Type: AWS::Serverless::Function
    Properties:
      PackageType: Image
      Architectures:
        - x86_64
      Events:
        # Lambdaをトリガーするイベント（EventBridge, API Gatewayなど）
        HelloWorld:
          Type: Api
          Properties:
            Path: /hello
            Method: get
            RestApiId: !Ref MyApi
    Metadata:
      Dockerfile: Dockerfile
      DockerContext: ./src
      DockerTag: python3.12-v1

  # （任意）ログの設定。ログもSAMで管理したいなら設定が必要
  # （自動生成のLogGroupだとStackの削除後も残る。参考：https://dev.classmethod.jp/articles/should-create-cloudwatch-logs-log-group-when-creating-lambda-with-aws-sam/）
  MyFunctionLogGroup:
    Type: AWS::Logs::LogGroup
    Properties:
      LogGroupName: !Sub /aws/lambda/${MyFunction}
      RetentionInDays: 14

Outputs:
  MyAPi:
    Description: "API Gateway endpoint URL"
    Value: !Sub "https://${MyApi}.execute-api.${AWS::Region}.amazonaws.com/Prod/hello/"
```
:::
